In [1]:
from omegaconf import OmegaConf
from datasets import load_data
config = OmegaConf.load("./configs/default_config.yaml")
for dataset in ['Imagenette', 'Flowers', 'EuroSAT', 'Aircraft', 'CIFAR10', 'CIFAR100' ]:
    config.dataset = dataset
    train_dataset, val_dataset, test_dataset, N, train_transform, transform = load_data(False, config) 
    print(dataset, len(train_dataset), len(val_dataset), len(test_dataset), N)

Imagenette 9469 2943 982 10
Flowers 1020 1020 6149 102
EuroSAT 18360 3240 5400 10
Aircraft 3334 3333 3333 100
Files already downloaded and verified
CIFAR10 47500 2500 10000 10
CIFAR100 47500 2500 10000 100
